In [1]:
import pandas as pd

#### Para testear las funciones, levanto los dataframes originales

In [255]:
universo = pd.read_csv(r'dataset.csv')
df_pageviews = pd.read_csv(r'df_pageviews.csv', parse_dates=['FEC_EVENT'])
df_conversiones = pd.read_csv(r'df_conversiones.csv')

In [256]:
df_pageviews['mes'] = df_pageviews['FEC_EVENT'].dt.month
df_pageviews['anio'] = df_pageviews['FEC_EVENT'].dt.year

In [263]:
df_page_qty = df_pageviews[df_pageviews['mes'].isin(qty_1)].groupby(['USER_ID', 'PAGE']).count()
df_page_qty = df_page_qty[['mes']]
df_page_qty = df_page_qty.reset_index()

,USER_ID,PAGE,mes
0,0,2,133
1,0,3,100
2,0,4,49
3,0,5,30
4,0,6,5
5,0,20,62
6,0,21,23
7,0,23,1
8,0,24,12
9,0,25,10


In [293]:
df = df_page_qty

aux = []
for page in df['PAGE'].unique():
    feature_name = 'page_{}_qty_9M'.format(page)
    df_feature = df[df.PAGE == page]
    df_feature.rename(columns={'mes': feature_name}, inplace=True)
    aux.append(df_feature)
aux = pd.concat(aux, axis=1)

In [323]:
page = df['PAGE'].unique()[0]
print('page: '+ str(page))
feature_name = 'page_{}_qty_9M'.format(page)
print('feature_name: '+ feature_name)
df_feature = df[df.PAGE == page]
df_feature.rename(columns={'mes': feature_name}, inplace=True)
df_feature.drop(['PAGE'], axis=1, inplace=True)
df_feature

df2 = pd.merge(universo, df_feature, how='left', left_on=['user_id'], right_on=['USER_ID'])
df2.drop(['USER_ID'], axis=1, inplace=True)


page: 2
feature_name: page_2_qty_9M


,user_id,snapshot_mes,target,page_2_qty_9M
0,0,9,0,133.0
1,1,9,0,26.0
2,2,9,0,33.0
3,3,9,0,41.0
4,4,9,0,55.0
5,5,9,0,37.0
6,6,9,0,26.0
7,7,9,0,46.0
8,8,9,0,22.0
9,9,9,0,48.0
